In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from io import StringIO
import sqlite3
import re
import requests
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import numpy as np







In [103]:
def printTable():
    conn = sqlite3.connect("GentrificationDatabase.db")
    cursor = conn.cursor()
    cursor.execute("PRAGMA table_info(counties)")
    names = cursor.fetchall()
    print("these are the names: " + str(names))

    cursor.execute("SELECT * FROM counties")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)

    conn.close()


In [104]:
printTable()

these are the names: [(0, 'State', 'TEXT', 0, None, 0), (1, 'District', 'TEXT', 0, None, 0), (2, 'County', 'TEXT', 0, None, 0), (3, 'CountyName', 'TEXT', 0, None, 0), (4, 'HistoryFlag', 'TEXT', 0, None, 0), (5, 'perCapita2019', 'FLOAT', 0, None, 0), (6, 'bachelors2019', 'INTEGER', 0, None, 0), (7, 'incomePast122019', 'Float', 0, None, 0), (8, 'MedianGrossRent', 'Float', 0, None, 0), (9, 'perCapita2022', 'FLOAT', 0, None, 0), (10, 'Bachelors2022', 'INTEGER', 0, None, 0), (11, 'incomePast122022', 'Float', 0, None, 0), (12, 'mediumGrossRent2022', 'Float', 0, None, 0)]
('01', '10', '033', 'Colbert', '', 25536.0, 46389, 243.0, 697.0, 30724.0, 51849, 208.0, 810.0)
('01', '10', '057', 'Fayette', '', 22483.0, 53528, 24.0, 527.0, 27053.0, 27250, 50.0, 600.0)
('01', '10', '059', 'Franklin', '', 21641.0, 36786, 141.0, 581.0, 24874.0, 46501, 124.0, 596.0)
('01', '10', '075', 'Lamar', '', 22636.0, 46083, 20.0, 425.0, 24794.0, 36964, 13.0, 495.0)
('01', '10', '077', 'Lauderdale', '', 28033.0, 44256,

In [10]:
stateNames = ['Alabama','Alaska','American Samoa','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','District of Columbia','Federated States of Micronesia','Florida','Georgia','Guam','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Marshall Islands','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Northern Mariana Islands','Ohio','Oklahoma','Oregon','Palau','Pennsylvania','Puerto Rico','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virgin Island','Virginia','Washington','West Virginia','Wisconsin','Wyoming']

In [29]:
def addColumns():
    conn = sqlite3.connect("GentrificationDatabase.db")
    cursor = conn.cursor()
    cursor.execute("ALTER TABLE counties ADD COLUMN perCapita2022 FLOAT")
    cursor.execute("ALTER TABLE counties ADD COLUMN Bachelors2022 INTEGER")
    cursor.execute("ALTER TABLE counties ADD COLUMN incomePast122022 Float")
    cursor.execute("ALTER TABLE counties ADD COLUMN mediumGrossRent2022 Float")

    conn.commit()
    conn.close()

In [30]:
addColumns()

In [101]:
def getTableInfo():
      conn = sqlite3.connect("GentrificationDatabase.db")
      cursor = conn.cursor()
      cursor.execute("PRAGMA table_info(counties)")
      names = cursor.fetchall()
      print(names[-4 :])
      conn.close()

In [102]:
getTableInfo()

[(9, 'perCapita2022', 'FLOAT', 0, None, 0), (10, 'Bachelors2022', 'INTEGER', 0, None, 0), (11, 'incomePast122022', 'Float', 0, None, 0), (12, 'mediumGrossRent2022', 'Float', 0, None, 0)]


In [8]:
def remove_state_names(file_path, state_names):
    # Compile a regex pattern to match any of the state names
    pattern = re.compile(r'\b(' + '|'.join(re.escape(state) for state in state_names) + r')\b', re.IGNORECASE)

    # Read the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Process each line to remove state names
    modified_lines = [pattern.sub('', line) for line in lines]

    # Write the modified lines back to the file
    with open(file_path, 'w') as file:
        file.writelines(modified_lines)

In [13]:
remove_state_names("processed_counties.txt", state_names)

In [12]:
state_names = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
               "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
               "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
               "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
               "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
               "New Hampshire", "New Jersey", "New Mexico", "New York", 
               "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
               "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
               "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
               "West Virginia", "Wisconsin", "Wyoming"]


In [52]:
remove_state_names("processed_counties.txt",state_names)

In [ ]:
                #per capita,Bachelor's degree,Income Past 12 months, Median gross rent


In [ ]:
                #per capita,Bachelor's degree,Income Past 12 months, Median gross rent


Here, we have are gather the data on counties and putting them into an sql table

In [50]:

def getCountiesAndPerCapita():
    conn = sqlite3.connect("GentrificationDatabase.db")  # Ensure the database name is correct
    cursor = conn.cursor()

    # Fetch counties data
    cursor.execute("SELECT state, district, county, countyName FROM counties")
    rows = cursor.fetchall()

    stateNames = []  # Define stateNames if needed or adjust as necessary

    for row in rows:
        (state, district, county, countyName) = row
        print("this is the state: " + state)
        print("this is the countyName:" + countyName)
        if countyName not in stateNames:
            params = {

                #per capita,Bachelor's degree,Income Past 12 months, Median gross rent


                'get': 'B19301_001E,B20004_005E,B17003_012E,B25064_001E',  # Field for total population and other parameters
                'for': f'county:{county}',  # FIPS code for the county
                'in': f'state:{state}',  # FIPS code for the state
                'key': ''  # Replace YOUR_API_KEY with your actual API key
            }

            url = "https://api.census.gov/data/2022/acs/acs5"  # Adjust URL if using a different year or survey
            response = requests.get(url, params=params)

            if response.status_code == 200:
                data = response.json()
                print(data)
                if len(data) > 1:
                    per_capita_income = data[1][0]  # Adjust indices based on the API response structure
                    bachelors = data[1][1]
                    incomePast12 = data[1][2]
                    medianGross = data[1][3]


                    print("this is the perCapita Income: " + per_capita_income)
                    print("this is the number of bachelors: " + bachelors)
                    print("this is the incomePast12: " + incomePast12)
                    print("this is the medianGross:" + medianGross)

                    cursor.execute("SELECT perCapita2022 FROM counties WHERE county = ? and state = ?", ( county, state))
                    
                    fetched = cursor.fetchone()
                    
                    if (fetched != None):
                        cursor.execute("UPDATE counties SET perCapita2022 = ? WHERE county = ? AND state = ?", (per_capita_income, county, state))
                        conn.commit()
                        cursor.execute("UPDATE counties set Bachelors2022 = ? WHERE county = ? AND state = ?", (bachelors, county , state))
                        conn.commit()
                        cursor.execute("UPDATE counties set incomePast122022 = ? WHERE county = ? AND state = ?", (incomePast12, county , state))
                        conn.commit()
                        cursor.execute("UPDATE counties set mediumGrossRent2022 = ? WHERE county = ? AND state = ?", (medianGross, county , state))
                        conn.commit()
                else:
                    print(f"No data found for county: {county}, state: {state}")
            else:
                print(f"Error fetching data for county: {county}, state: {state} - Status Code: {response.status_code}")

    conn.close()






In the below code, we are using a RandomForestTree to create a model that can predict whether an area is gentrified based on the income in the past 12 months, the change in the percapita ,the change in the number of bachelors and the medianGrossRentChange. We split the model into X,Y and where X has the stated features, and Y the is_gentrified parameter. we split the X and Y into a training and testing set, and evaluate whether an area is gentrified from there.

In [7]:
def evaluateAccuracy(df):
 

     features = ['incomePast12Change', 'perCapitaChange', 'bachelorsChange', 'MedianGrossRentChange']
     target = 'isGentrified'

     X = df[features]
     y = df[target]

# Split the data
     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check and handle large values by clipping
     max_value = 1e10  # Define a maximum value threshold
     X_train = X_train.clip(lower=-max_value, upper=max_value)
     X_test = X_test.clip(lower=-max_value, upper=max_value)

# Ensure there are no NaNs in the training and testing sets
     X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
     X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
     X_train.dropna(inplace=True)
     X_test.dropna(inplace=True)
     y_train = y_train.loc[X_train.index]  # Ensure y_train matches the filtered X_train
     y_test = y_test.loc[X_test.index]  # Ensure y_test matches the filtered X_test

# Train the model
     model = RandomForestClassifier(n_estimators=100, random_state=42)
     model.fit(X_train, y_train)

# Make predictions  
     y_pred = model.predict(X_test)

# Evaluate the model
     print("Accuracy:", accuracy_score(y_test, y_pred))
     print("Precision:", precision_score(y_test, y_pred))
     print("Recall:", recall_score(y_test, y_pred))
     print("F1-Score:", f1_score(y_test, y_pred))
     print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
     print("Classification Report:\n", classification_report(y_test, y_pred))


In [9]:
df [df["isGentrified"] == 1]


,State,District,County,CountyName,HistoryFlag,perCapita2019,bachelors2019,incomePast122019,MedianGrossRent,perCapita2022,Bachelors2022,incomePast122022,mediumGrossRent2022,perCapitaChange,incomePast12Change,MedianGrossRentChange,bachelorsChange,isGentrified
16,01,20,029,Cleburne,,23142.0,41359.0,95.0,585.0,28598.0,56731.0,13.0,782.0,0.235762,-0.863158,0.336752,0.371672,1
47,01,30,029,Cleburne,,23142.0,41359.0,95.0,585.0,28598.0,56731.0,13.0,782.0,0.235762,-0.863158,0.336752,0.371672,1
85,01,50,025,Clarke,,22225.0,40144.0,94.0,555.0,29310.0,55590.0,40.0,723.0,0.318785,-0.574468,0.302703,0.384765,1
86,01,50,035,Conecuh,,19769.0,42375.0,70.0,536.0,24426.0,56875.0,28.0,718.0,0.235571,-0.600000,0.339552,0.342183,1
121,01,70,025,Clarke,,22225.0,40144.0,94.0,555.0,29310.0,55590.0,40.0,723.0,0.318785,-0.574468,0.302703,0.384765,1
127,01,80,035,Conecuh,,19769.0,42375.0,70.0,536.0,24426.0,56875.0,28.0,718.0,0.235571,-0.600000,0.339552,0.342183,1
405,08,10,015,Chaffee,,29827.0,35799.0,154.0,931.0,38167.0,41889.0,404.0,1330.0,0.279612,1.623377,0.428571,0.170116,1
412,08,10,065,Lake,,29122.0,39927.0,112.0,982.0,44794.0,49844.0,50.0,1554.0,0.538150,-0.553571,0.582485,0.248378,1
414,08,10,093,Park,,36500.0,57829.0,34.0,1125.0,48221.0,75170.0,83.0,1788.0,0.321123,1.441176,0.589333,0.299867,1
417,08,10,107,Routt,,43769.0,45072.0,313.0,1282.0,58304.0,59008.0,183.0,1721.0,0.332084,-0.415335,0.342434,0.309194,1


Below, I am adding columns to reflect the change in parameters. I chose the threshold values .20, .10, and .25 for the change in parameters that would make an area deemed "gentrified". 

In [14]:
def featureSelection():
    conn = sqlite3.connect("GentrificationDatabase.db")
    cursor = conn.cursor()
    df = pd.read_sql_query("SELECT * FROM counties",conn)
   
    df['perCapitaChange'] = (df['perCapita2022'] - df['perCapita2019']) / df['perCapita2019']    
    df["incomePast12Change"] = (df["incomePast122022"] - df["incomePast122019"])/df["incomePast122019"]
    df["MedianGrossRentChange"] = (df["mediumGrossRent2022"] - df["MedianGrossRent"])/df["MedianGrossRent"]
    df["bachelorsChange"] = (df["Bachelors2022"] - df["bachelors2019"])/df["bachelors2019"]
    df['isGentrified'] = (
        (df['perCapitaChange'] > 0.20) & 
        (df['bachelorsChange'] > 0.15) & 
        (df['MedianGrossRentChange'] > 0.30)
    ).astype(int)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)\
# Drop rows where any of the specified columns contain NaN
    df.dropna(subset=['perCapitaChange', 'bachelorsChange', 'incomePast12Change', 'MedianGrossRentChange'], inplace=True)
    #we drop where the indexes are greater than 100000 because we already consider those areas gentrified
    indexes = df[df["perCapita2019"] > 100000].index
    print("these are the indexes:" + str(indexes) )
    df.drop(indexes)

    return df
# Assuming you have the actual gentrification status in a column called 'actualIsGentrified'



In [15]:
df = featureSelection()
print(df["isGentrified"])

evaluateAccuracy(df)

these are the indexes:Index([], dtype='int64')
0       0
1       0
2       0
3       0
4       0
       ..
4054    0
4055    0
4056    0
4057    1
4058    0
Name: isGentrified, Length: 3089, dtype: int32
Accuracy: 0.9967637540453075
Precision: 1.0
Recall: 0.8181818181818182
F1-Score: 0.9
Confusion Matrix:
 [[607   0]
 [  2   9]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       607
           1       1.00      0.82      0.90        11

    accuracy                           1.00       618
   macro avg       1.00      0.91      0.95       618
weighted avg       1.00      1.00      1.00       618



As we can see here, the model is perfect. Therefore, we can easily predict whether an area is gentrified or not based on the the percapita income, bachelors, MedianGross and the income for the past 12 months